<a href="https://colab.research.google.com/github/DionesGouvea/DeepLearning.AI/blob/main/Generative%20AI%20with%20Large%20Language%20Models/Lab_1_summarize_dialogue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Caso de uso de IA generativa: sumarizar o diálogo

Bem-vindo ao lado prático deste curso. Neste laboratório você realizará a tarefa de resumo de diálogos usando IA generativa. Você explorará como o texto de entrada afeta a saída do modelo e realizará engenharia imediata para direcioná-lo para a tarefa necessária. Ao comparar inferências de disparo zero, de disparo único e de poucos disparos, você dará o primeiro passo em direção à engenharia imediata e verá como ela pode aprimorar a produção generativa de modelos de linguagem grande.

# Table of Contents

- [ 1 - Configurar o Kernel e Dependências Necessárias](#1)
- [ 2 - Sumarizar Diálogo sem Engenharia de Prompt](#2)
- [ 3 - Sumarizar Diálogo com um Prompt de Instrução](#3)
  - [ 3.1 - Inferência de Zero Shot com um Prompt de Instrução](#3.1)
  - [ 3.2 - Inferência de Zero Shot com o Modelo de Prompt do FLAN-T5](#3.2)
- [ 4 - Sumarizar Diálogo com Inferência de One Shot e Few Shot](#4)
  - [ 4.1 - Inferência de One Shot](#4.1)
  - [ 4.2 - Inferência de Few Shot](#4.2)
- [ 5 - Parâmetros de Configuração Generativa para Inferência](#5)


<a name='1'></a>
## 1 - Set up Kernel and Required Dependencies (SAGE MAKER ONLY)

First, check that the correct kernel is chosen.

<img src="images/kernel_set_up.png" width="300"/>

You can click on that (top right of the screen) to see and check the details of the image, kernel, and instance type.

<img src="images/w1_kernel_and_instance_type.png" width="600"/>




In [ ]:
#comentado por ser usado somente no sagemaker

'''import os  # Importa o módulo os para acessar variáveis de ambiente do sistema operacional

# Define a instância esperada com o tipo de instância desejado
instance_type_expected = 'ml-m5-2xlarge'

# Obtém o tipo de instância atual do ambiente
instance_type_current = os.environ.get('HOSTNAME')

# Imprime o tipo de instância esperado
print(f'Expected instance type: instance-datascience-{instance_type_expected}')

# Imprime o tipo de instância atual
print(f'Currently chosen instance type: {instance_type_current}')

# Verifica se o tipo de instância esperado está presente no tipo de instância atual
assert instance_type_expected in instance_type_current, f'ERROR. You selected the {instance_type_current} instance type. Please select {instance_type_expected} instead as shown on the screenshot above'

# Se a verificação passar, imprime que o tipo de instância foi escolhido corretamente
print("Instance type has been chosen correctly.")
'''

Now install the required packages to use PyTorch and Hugging Face transformers and datasets.



In [1]:
# Instala a versão 2.17.0 da biblioteca 'datasets' utilizando o gerenciador de pacotes pip
%pip install -U datasets==2.17.0

# Atualiza o próprio pip para a versão mais recente
%pip install --upgrade pip

# Instala as bibliotecas torch e torchdata com as versões específicas 1.13.1 e 0.5.1, respectivamente,
# e desativa a verificação da versão do pip durante a instalação
%pip install --disable-pip-version-check torch==1.13.1 torchdata==0.5.1 --quiet

# Instala a biblioteca transformers com a versão específica 4.27.2
%pip install transformers==4.27.2 --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 75.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 54.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.

Carregue os conjuntos de dados, Large Language Model (LLM), tokenizer e configurador. Não se preocupe se você ainda não entendeu todos esses componentes - eles serão descritos e discutidos posteriormente neste caderno.

In [2]:
from datasets import load_dataset
from transformers import AutoModelForSeq2SeqLM
from transformers import AutoTokenizer
from transformers import GenerationConfig

<a name='2'></a>
## 2 - Sumariza um diálogo sem engenharia de prompt


Neste caso de uso, será gerado um resumo de um diálogo com o modelo de linguagem pré-treinado Large Language Model (LLM) FLAN-T5 da Hugging Face.

A lista de modelos disponíveis no pacote transformers da Hugging Face pode ser encontrada aqui: [here](https://huggingface.co/docs/transformers/index).

 [DialogSum](https://huggingface.co/datasets/knkarthick/dialogsum)

In [3]:
# Carrega alguns diálogos simples do conjunto de dados DialogSum da Hugging Face.
# Este conjunto de dados contém 10.000+ diálogos com os resumos e tópicos manualmente rotulados correspondentes.

huggingface_dataset_name = "knkarthick/dialogsum"

# Carrega o conjunto de dados utilizando a função load_dataset do pacote Hugging Face
dataset = load_dataset(huggingface_dataset_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Imprime alguns diálogos com seus resumos de linha de base

In [4]:
# Define os índices dos exemplos que serão impressos
example_indices = [40, 200]

# Cria uma linha de traços para separar as seções
dash_line = '-'.join('' for x in range(100))


for i, index in enumerate(example_indices):
    # Imprime uma linha de traços para separar os exemplos
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)

    # Imprime o diálogo de entrada
    print('INPUT DIALOGUE:')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)

    # Imprime o resumo humano de linha de base
    print('BASELINE HUMAN SUMMARY:')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()


---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT DIALOGUE:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
Exa

Carrega o modelo [FLAN-T5 model](https://huggingface.co/docs/transformers/model_doc/flan-t5), criando uma instância da classe `AutoModelForSeq2SeqLM` com o método `.from_pretrained()`

In [6]:
model_name = 'google/flan-t5-base'

# Carrega o modelo utilizando o método .from_pretrained() da classe AutoModelForSeq2SeqLM
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Para realizar a codificação e decodificação, é necessário trabalhar com texto em forma tokenizada.
A tokenização é o processo de dividir textos em unidades menores que podem ser processadas pelos modelos LLM.

Baixa o tokenizer para o modelo FLAN-T5 usando o método AutoTokenizer.`from_pretrained()` O parâmetro use_fast ativa o tokenizer rápido. Neste estágio, não é necessário entrar nos detalhes disso, mas você pode encontrar os parâmetros do tokenizer na documentação. [documentation](https://huggingface.co/docs/transformers/v4.28.1/en/model_doc/auto#transformers.AutoTokenizer).

In [7]:
# Baixa o tokenizer utilizando o método .from_pretrained() da classe AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [8]:
# Testa a codificação e decodificação do tokenizer com uma frase simples:

sentence = "What time is it, Tom?"

# Codifica a frase usando o tokenizer
sentence_encoded = tokenizer(sentence, return_tensors='pt')

# Decodifica a frase utilizando o tokenizer
sentence_decoded = tokenizer.decode(
        sentence_encoded["input_ids"][0],
        skip_special_tokens=True
    )

# Imprime a frase codificada e decodificada
print('ENCODED SENTENCE:')
print(sentence_encoded["input_ids"][0])
print('\nDECODED SENTENCE:')
print(sentence_decoded)


ENCODED SENTENCE:
tensor([ 363,   97,   19,   34,    6, 3059,   58,    1])

DECODED SENTENCE:
What time is it, Tom?


Agora é hora de explorar o quão bem o LLM base resume um diálogo sem engenharia de prompt.

A engenharia de prompt é um ato de um humano alterando o prompt (entrada) para melhorar a resposta para uma determinada tarefa.


In [9]:
# Loop sobre os índices dos exemplos e os enumera
for i, index in enumerate(example_indices):
    # Obtém o diálogo do conjunto de dados de teste com o índice especificado
    dialogue = dataset['test'][index]['dialogue']

    # Obtém o resumo do conjunto de dados de teste com o índice especificado
    summary = dataset['test'][index]['summary']


    # Codifica o diálogo utilizando o tokenizer
    inputs = tokenizer(dialogue, return_tensors='pt')

    # Gera a saída do modelo sem engenharia de prompt
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    # Imprime as informações do exemplo
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{dialogue}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{output}\n')

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - WITHOUT PROMPT ENGINEERING:
Person1: It's ten to nine.

-------------------------------

Você pode ver que os palpites do modelo fazem algum sentido, mas não parece ter certeza sobre qual tarefa deve realizar. Parece que ele apenas inventa a próxima frase no diálogo. A engenharia de prompts pode ajudar aqui.

<a name='3'></a>
## 3 - Summarizando o Dialogo com um prompt de instruções

A engenharia de prompt é um conceito importante no uso de modelos básicos para geração de texto. Você pode conferir [este blog](https://www.amazon.science/blog/emnlp-prompt-engineering-is-the-new-feature-engineering) da Amazon Science para uma rápida introdução à engenharia imediata

<a name='3.1'></a>
### 3.1 - Zero Shot Inferência com um prompt de instrução

Para instruir o modelo a executar uma tarefa – resumir um diálogo – você pode pegar o diálogo e convertê-lo em um prompt de instrução. Isso geralmente é chamado de **zero shot inference**.  Você pode conferir [este blog da AWS](https://aws.amazon.com/blogs/machine-learning/zero-shot-prompting-for-the-flan-t5-foundation-model-in-amazon-sagemaker-jumpstart/) para uma rápida descrição do que é aprendizagem zero shot e por que é um conceito importante para o modelo LLM.

Envolva o diálogo em uma instrução descritiva e veja como o texto gerado mudará:

In [10]:
# Loop sobre os índices dos exemplos e os enumera
for i, index in enumerate(example_indices):
    # Obtém o diálogo do conjunto de dados de teste com o índice especificado
    dialogue = dataset['test'][index]['dialogue']

    # Obtém o resumo do conjunto de dados de teste com o índice especificado
    summary = dataset['test'][index]['summary']

    # Constrói o prompt com o diálogo
    prompt = f"""
Summarize the following conversation.

{dialogue}

Summary:
    """

    # Input constructed prompt instead of the dialogue.
    # Codifica o prompt usando o tokenizer
    inputs = tokenizer(prompt, return_tensors='pt')

    # Gera a saída do modelo
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    # Imprime as informações do exemplo
    print(dash_line)
    print('Example ', i + 1)
    print(dash_line)
    print(f'INPUT PROMPT:\n{prompt}')
    print(dash_line)
    print(f'BASELINE HUMAN SUMMARY:\n{summary}')
    print(dash_line)
    print(f'MODEL GENERATION - ZERO SHOT:\n{output}\n')


---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
INPUT PROMPT:

Summarize the following conversation.

#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

Summary:
    
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
MODEL GENERATION - ZERO SHOT:
The train is about to

Isto é muito melhor! Mas o modelo ainda não capta as nuances das conversas.

**Exercício:**

- Experimente o texto `prompt` e veja como as inferências serão alteradas. As inferências mudarão se você encerrar o prompt apenas com uma string vazia em vez de `Resumo: `?
- Tente reformular o início do texto do `prompt` de `Resuma a seguinte conversa.` para algo diferente - e veja como isso influenciará a saída gerada.

<a nome='3.2'></a>
### 3.2 - Inferência Zero Shot com o modelo de prompt do FLAN-T5

Vamos usar um prompt um pouco diferente. O FLAN-T5 tem muitos modelos de prompt publicados para determinadas tarefas [aqui](https://github.com/google-research/FLAN/tree/main/flan/v2). No código a seguir, você usará um dos [prompts FLAN-T5 pré-construídos](https://github.com/google-research/FLAN/blob/main/flan/v2/templates.py):

In [11]:
# Para cada exemplo no conjunto de dados de teste
for i, index in enumerate(example_indices):
    # Obtenha o diálogo e o resumo do exemplo
    dialogue = dataset['test'][index]['dialogue']
    summary = dataset['test'][index]['summary']

    # Crie um prompt com o diálogo
    prompt = f"""
    Dialogue:

    {dialogue}

    What was going on?
    """

    # Codifique o prompt para entrada do modelo
    inputs = tokenizer(prompt, return_tensors='pt')

    # Gere uma saída do modelo
    output = tokenizer.decode(
        model.generate(
            inputs["input_ids"],
            max_new_tokens=50,
        )[0],
        skip_special_tokens=True
    )

    # Imprima os resultados
    print(dash_line)
    print('Exemplo ', i + 1)
    print(dash_line)
    print(f'PROMPT DE ENTRADA:\n{prompt}')
    print(dash_line)
    print(f'RESUMO HUMANO DE REFERÊNCIA:\n{summary}\n')
    print(dash_line)
    print(f'GERAÇÃO DO MODELO - ZERO SHOT:\n{output}\n')


---------------------------------------------------------------------------------------------------
Exemplo  1
---------------------------------------------------------------------------------------------------
PROMPT DE ENTRADA:

    Dialogue:

    #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

    What was going on?
    
---------------------------------------------------------------------------------------------------
RESUMO HUMANO DE REFERÊNCIA:
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.

---------------------------------------------------------------------------------------------------
GERAÇÃO DO MODELO - ZERO SHOT:
Tom is late for


Observe que esse prompt do FLAN-T5 ajudou um pouco, mas ainda tem dificuldade para captar as nuances da conversa. Isso é o que você tentará resolver com algumas inferências.

<a nome='4'></a>
## 4 - Resuma o diálogo com inferência de um tiro e de poucos tiros

**One shot and few shot inference** são as práticas de fornecer a um LLM um ou mais exemplos completos de pares de prompt-resposta que correspondem à sua tarefa - antes do prompt real que você deseja concluir. Isso é chamado de “aprendizado em contexto” e coloca seu modelo em um estado que compreende sua tarefa específica. Você pode ler mais sobre isso [neste blog do HuggingFace](https://huggingface.co/blog/few-shot-learning-gpt-neo-and-inference-api).

<a name='4.1'></a>
### 4.1 - One Shot Inference

Vamos construir uma função que pega uma lista de `example_indices_full`, gera um prompt com exemplos completos e, no final, anexa o prompt que você deseja que o modelo complete (`example_index_to_summarize`). Você usará o mesmo modelo de prompt FLAN-T5 da seção [3.2](#3.2).

In [12]:
# Definindo uma função chamada "make_prompt" que recebe dois argumentos:
# - example_indices_full: uma lista de índices de exemplos completos
# - example_index_to_summarize: o índice do exemplo que desejamos resumir

def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''  # Inicializa uma string vazia para armazenar o prompt

    # Itera sobre os índices dos exemplos completos
    for index in example_indices_full:
        # Obtém o diálogo e o resumo do exemplo
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']

        # A sequência de parada '{summary}\n\n\n' é importante para o modelo FLAN-T5.
        # Outros modelos podem ter sua própria sequência de parada preferida.
        prompt += f"""
        Dialogue:

        {dialogue}

        What was going on?
        {summary}


        """

    # Obtém o diálogo do exemplo que desejamos resumir
    dialogue = dataset['test'][example_index_to_summarize]['dialogue']

    # Adiciona o diálogo ao prompt
    prompt += f"""
    Dialogue:

    {dialogue}

    What was going on?
    """

    return prompt  # Retorna o prompt completo


Construa o prompt para realizar  shot inference:

In [13]:
# Definindo duas listas de índices de exemplos
example_indices_full = [40]  # Índices dos exemplos completos
example_index_to_summarize = 200  # Índice do exemplo que desejamos resumir

# Criação do prompt usando a função "make_prompt"
one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

# Imprime o prompt gerado
print(one_shot_prompt)



        Dialogue:

        #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

        What was going on?
        #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.


        
    Dialogue:

    #Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a 


Agora passe este prompt para realizar a shot inference:

In [14]:
# Obtém o resumo do exemplo que desejamos resumir
summary = dataset['test'][example_index_to_summarize]['summary']

# Codifica o prompt para entrada do modelo
inputs = tokenizer(one_shot_prompt, return_tensors='pt')

# Gera uma saída do modelo
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

# Imprime os resultados
print(dash_line)
print(f'RESUMO HUMANO DE REFERÊNCIA:\n{summary}\n')
print(dash_line)
print(f'GERAÇÃO DO MODELO - ONE SHOT:\n{output}')


---------------------------------------------------------------------------------------------------
RESUMO HUMANO DE REFERÊNCIA:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
GERAÇÃO DO MODELO - ONE SHOT:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.


<a name='4.2'></a>
### 4.2 - Few Shot Inference

Vamos explorar a inferência de algumas cenas adicionando mais dois pares completos de resumo de diálogo ao seu prompt.

In [28]:
# Definindo uma lista de índices de exemplos completos
example_indices_full = [45, 85, 125, 145, 175]

# Definindo o índice do exemplo que desejamos resumir
example_index_to_summarize = 200

# Criação do prompt usando a função "make_prompt"
few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

# Imprime o prompt gerado
print(few_shot_prompt)



        Dialogue:

        #Person1#: Would you like to go to the party tonight?
#Person2#: Whose party?
#Person1#: Ruojia's. Don't you know that? Ruojia has got married.
#Person2#: What! Is she really? I can't believe it!
#Person1#: Yes. Yesterday.
#Person2#: Good gracious. That's incredible! I feel so happy for her!
#Person1#: Yes, me too.
#Person2#: But how do you know that?
#Person1#: I saw the news from her twitter. And she sent an email about it.
#Person2#: What? I didn't receive it!
#Person1#: Maybe you should check your email.
#Person2#: Oh yes, I find it. Tonight at her home. Will you bring something?
#Person1#: Yes, a pair of wineglasses and a card to wish her happy marriage.
#Person2#: I will buy a tea set.

        What was going on?
        #Person1# tells #Person2# that Ruojia is married and will have a party tonight. #Person2#'s surprised to know that. They will bring their gifts to bless her.


        
        Dialogue:

        #Person1#: Stupid girl, making me spend

Agora passe este prompt para realizar few shot inference:

In [29]:
# Obtém o resumo do exemplo que desejamos resumir
summary = dataset['test'][example_index_to_summarize]['summary']

# Codifica o prompt para entrada do modelo
inputs = tokenizer(few_shot_prompt, return_tensors='pt')

# Gera uma saída do modelo
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        max_new_tokens=50,
    )[0],
    skip_special_tokens=True
)

# Imprime os resultados
print(dash_line)
print(f'RESUMO HUMANO DE REFERÊNCIA:\n{summary}\n')
print(dash_line)
print(f'GERAÇÃO DO MODELO - FEW SHOT:\n{output}')


---------------------------------------------------------------------------------------------------
RESUMO HUMANO DE REFERÊNCIA:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
GERAÇÃO DO MODELO - FEW SHOT:
#Person1 recommends upgrading the system and hardware.


Neste caso, poucos disparos não proporcionaram muita melhoria em relação à inferência de um disparo. E qualquer coisa acima de 5 ou 6 doses normalmente também não ajuda muito. Além disso, você precisa ter certeza de não exceder o comprimento do contexto de entrada do modelo que, em nosso caso, é de 512 tokens. Qualquer coisa acima do comprimento do contexto será ignorada.

No entanto, você pode ver que fornecer pelo menos um exemplo completo (one shot) fornece ao modelo mais informações e melhora qualitativamente o resumo geral.

**Exercício:**

Experimente Few shot inference.
- Escolha diferentes diálogos - altere os índices na lista `example_indices_full` e o valor `example_index_to_summarize`.
- Altere o número de tiros. No entanto, certifique-se de permanecer dentro do comprimento de contexto 512 do modelo.

Quão bem few inference funciona com outros exemplos?

<a name='5'></a>
## 5 - Generative Configuration Parameters for Inference

Você pode alterar os parâmetros de configuração do método `generate()` para ver uma saída diferente do LLM. Até agora o único parâmetro que você configurou foi `max_new_tokens=50`, que define o número máximo de tokens a serem gerados. Uma lista completa de parâmetros disponíveis pode ser encontrada na [documentação do Hugging Face Generation](https://huggingface.co/docs/transformers/v4.29.1/en/main_classes/text_generation#transformers.GenerationConfig).

Uma maneira conveniente de organizar os parâmetros de configuração é usar a classe `GenerationConfig`.

**Exercício:**

Altere os parâmetros de configuração para investigar sua influência na saída.

Colocando o parâmetro `do_sample = True`, você ativa várias estratégias de decodificação que influenciam o próximo token da distribuição de probabilidade em todo o vocabulário. Você pode então ajustar as saídas alterando a `temperatura` e outros parâmetros (como `top_k` e `top_p`).

Remova o comentário das linhas da célula abaixo e execute novamente o código. Tente analisar os resultados. Você pode ler alguns comentários abaixo.

In [31]:
#generation_config = GenerationConfig(max_new_tokens=50)
#generation_config = GenerationConfig(max_new_tokens=10)
generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.3,top_k=5)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=0.5)
# generation_config = GenerationConfig(max_new_tokens=50, do_sample=True, temperature=1.0)

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(
        inputs["input_ids"],
        generation_config=generation_config,
    )[0],
    skip_special_tokens=True
)

print(dash_line)
print(f'MODEL GENERATION - FEW SHOT:\n{output}')
print(dash_line)
print(f'BASELINE HUMAN SUMMARY:\n{summary}\n')

---------------------------------------------------------------------------------------------------
MODEL GENERATION - FEW SHOT:
#Person1 recommends upgrading their system and hardware.
---------------------------------------------------------------------------------------------------
BASELINE HUMAN SUMMARY:
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.



Comentários relacionados à escolha dos parâmetros na célula de código acima:
- Escolher `max_new_tokens=10` tornará o texto de saída muito curto, então o resumo do diálogo será cortado.
- Colocando `do_sample = True` e alterando o valor da temperatura você obtém mais flexibilidade na saída.